Reference: https://developer.nvidia.com/gpugems/gpugems3/part-vi-gpu-computing/chapter-39-parallel-prefix-sum-scan-cuda  
https://bbs.huaweicloud.com/blogs/323350

prefix_scan_0_with_sysn (Hillis-Steele): used __gpu_sync function in kernel function for syschronization. It needs extra global variable and sleep function to barrier all block.  
Another way: used other global variable to achieve offset addition in all blocks.  
prefix_scan_0 (Hillis-Steele)

## Hillis-Steele Prefix Scan Algorithm

### Initial State

Before any operations, the output array is initialized to:

| Index | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
| Value | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |

### Step 1: s = 1

Each element adds the value of its immediate left neighbor (**i - 1**):

| Index  | 0      | 1      | 2      | 3      | 4      | 5      | 6      | 7      |
|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
| Operation | -      | 0 + 1  | 1 + 2  | 2 + 3  | 3 + 4  | 4 + 5  | 5 + 6  | 6 + 7  |
| Result | 0      | 1      | 3      | 5      | 7      | 9      | 11     | 13     |


### Step 2: s = 2

Each element adds the value two positions to its left (**i - 2**):

| Index  | 0 | 1 | 2        | 3        | 4        | 5        | 6        | 7        |
|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
| Operation | - | - | 0 + (1 + 2) | (0 + 1) + (2 + 3) | (1 + 2) + (3 + 4) | (2 + 3) + (4 + 5) | (3 + 4) + (5 + 6) | (4 + 5) + (6 + 7) |
| Result | 0 | 1 | 3        | 6        | 10       | 14       | 18       | 22       |

### Step 3: s = 4

Each element adds the value four positions to its left (**i - 4**):

| Index  | 0 | 1 | 2 | 3 | 4                | 5                | 6                | 7                |
|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
| Operation | - | - | - | - | 0 + (1 + 2 + 3 + 4) | (0 + 1) + (2 + 3 + 4 + 5) | (0 + 1 + 2) + (3 + 4 + 5 + 6)| (0 + 1 + 2 + 3) + (4 + 5 + 6 + 7) |
| Result | 0 | 1 | 3 | 6 | 10               | 15               | 21               | 28               |


In [ ]:
%%writefile prefix_scan.cu
#include <stdio.h>
#include <stdlib.h>
#include <type_traits>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/sequence.h>
// #include <cooperative_groups.h>
// #include <cuda_runtime_api.h>


#define TYPE int
#define N 20240
#define BLOCK_SIZE 1024
#define NUM_PER_THREAD 8
#define WARP_SIZE (BLOCK_SIZE / NUM_PER_THREAD / 32)

// namespace cg = cooperative_groups;

// lock-based
__device__ volatile int g_mutex[1024];

// GPU lock-based synchronization function
__device__ void __gpu_sync(int goalVal)
{
  // thread ID in a block
  int tid_in_block = threadIdx.x * blockDim.y + threadIdx.y;
  int bid_in_grid = blockIdx.x * gridDim.y + blockIdx.y;

  // only thread 0 is used for synchronization
  if (tid_in_block == 0)
  {
    for(int i = 0; i < gridDim.x * gridDim.y; ++i)
      atomicAdd((int*) &(g_mutex[i]), 1);

    // only when all blocks add 1 to g_mutex
    // will g_mutex equal to goalVal
    while (g_mutex[bid_in_grid] != goalVal)
    {
      // Do nothing here
    }
    g_mutex[bid_in_grid] = 0;
  }
  __syncthreads();

  unsigned int count = 0;
  while (count < 100) {
      count++;
  }
}


__global__ void  warm_up()
{
    int indexX = threadIdx.x + blockIdx.x * blockDim.x;
    if (indexX < N)
    {
        float a = 0.0f;
        float b = 1.0f;
        float c = a + b;
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
void prefix_scan(T* input, T* output, int n)
{
    output[0] = input[0];
    for (int i = 1; i < n; ++i)
    {
        output[i] = output[i - 1] + input[i];
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void prefix_scan_0_with_sysn(T* input, T* output, int n)
{
    extern __shared__ T sdata[];

    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int tid = threadIdx.x;

    if (idx < n)
    {
        sdata[tid] = input[idx];
    }
    else
    {
        sdata[tid] = 0;
    }

    __syncthreads();

    for (unsigned int s = 1; s < blockDim.x; s *= 2)
    {
        T temp = 0;
        if (tid >= s)
        {
            temp = sdata[tid - s];
        }
        __syncthreads();
        sdata[tid] += temp;
        __syncthreads();
    }

    if (idx < n)
    {
        output[idx] = sdata[tid];
    }

    // Get the thread block and grid
    // cg::thread_block cta = cg::this_thread_block();
    // cg::grid_group grid = cg::this_grid();

    printf("blockIdx.x: %d, gridDim.x: %d\n", blockIdx.x, gridDim.x);
    for(int i = 1; i < gridDim.x*gridDim.y; ++i)
    {
        // __threadfence();// It doesn't work
        // grid.sync();// It doesn't work
        __gpu_sync(gridDim.x*gridDim.y);
        T offset = output[i * blockDim.x - 1];
        if (idx < n && blockIdx.x == i)
        {
            output[idx] += offset;
            printf("i: %d idx: %d blockIdx.x: %d, gridDim.x: %d, output: %d\n",i, idx, blockIdx.x, gridDim.x, output[idx]);
        }

        // __threadfence();
        // grid.sync(); // It doesn't work
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void prefix_scan_0(T* input, T* output, int n)
{
    extern __shared__ T sdata[];

    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int tid = threadIdx.x;

    if (idx < n)
    {
        sdata[tid] = input[idx];
    }
    else
    {
        sdata[tid] = 0;
    }

    for (unsigned int s = 1; s < blockDim.x; s *= 2)
    {
        __syncthreads();
        T temp = 0;
        if (tid >= s)
        {
            temp = sdata[tid - s];
        }
        __syncthreads();
        sdata[tid] += temp;

    }

    if (idx < n)
    {
        output[idx] = sdata[tid];
        // printf("idx: %d blockIdx.x: %d, output: %d\n",idx, blockIdx.x, output[idx]);
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void cal_offset(T* output, T* offset)
{
    int tid = threadIdx.x;
    int bid = blockIdx.x;

    offset[bid] = 0;

    if (tid == 0)
    {
        for(int i = 1; i <= blockIdx.x; ++i)
        {
          offset[bid] += output[i * blockDim.x - 1];
        }
        // printf("offset[bid], %d, bid, %d\n", offset[bid], bid);
    }

}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void add_offset(T* output, T* offset, int n)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n)
    {
        output[idx] += offset[blockIdx.x];
    }
}

int main()
{
    TYPE h_input[N];
    for (int i = 0; i < N; ++i)
    {
        h_input[i] = 1;
    }

    TYPE h_output[N];
    prefix_scan(h_input, h_output, N);


    thrust::device_vector<TYPE> d_input(h_input, h_input + N);
    thrust::device_vector<TYPE> d_output(N);

    int threads_per_block = BLOCK_SIZE;
    int no_of_blocks = (N + threads_per_block - 1) / threads_per_block;

    warm_up<<<no_of_blocks, threads_per_block>>>();
    prefix_scan_0<<<no_of_blocks, threads_per_block, threads_per_block * sizeof(TYPE)>>>(thrust::raw_pointer_cast(d_input.data()), thrust::raw_pointer_cast(d_output.data()), int(N));

    thrust::device_vector<TYPE> d_offset(no_of_blocks);
    cal_offset<<<no_of_blocks, threads_per_block>>>(thrust::raw_pointer_cast(d_output.data()), thrust::raw_pointer_cast(d_offset.data()));
    add_offset<<<no_of_blocks, threads_per_block>>>(thrust::raw_pointer_cast(d_output.data()), thrust::raw_pointer_cast(d_offset.data()), int(N));

    cudaDeviceSynchronize();

    thrust::host_vector<TYPE> h_result = d_output;

    bool match = true;
    for (int i = 0; i < N; ++i)
    {
        if (h_output[i] != h_result[i])
        {
            std::cout << i << " " << h_output[i] << " " << h_result[i] << std::endl;
            match = false;
            break;
        }
    }

    if (match)
        std::cout << "Results match!" << std::endl;
    else
        std::cout << "Results do not match!" << std::endl;

    return 0;
}

In [ ]:
!nvcc -o prefix_scan -lineinfo prefix_scan.cu

In [ ]:
!./prefix_scan